In [2]:
import boto3
import pandas as pd
import glob
import os
import json
import xmltodict
import sys
from collections import Counter
import numpy as np
from spellchecker import SpellChecker
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
fn = glob.glob(os.path.join('.', '*final.json'))[0]
fr = glob.glob(os.path.join('results3', '*final.json'))[0]


In [4]:
fn

'./created_2019-Jan-29_18_22_44_final.json'

In [5]:
fr

'results3/created_2019-Jan-29_18_22_44_final.json'

In [6]:
with open(fn, 'r') as handle:
    hitlist_a = json.load(handle)
with open(fr, 'r') as handle:
    resultlist_a = json.load(handle)
print(len(resultlist_a))
print(len(hitlist_a))
#print(hitlist_a)

4
4


In [7]:
fn = glob.glob(os.path.join('../../pilot01b2019/production', '*final.json'))[0]
fr = glob.glob(os.path.join('../../pilot01b2019/production/results3', '*final.json'))[0]
print(fn)
with open(fn, 'r') as handle:
    hitlist_b = json.load(handle)
with open(fr, 'r') as handle:
    resultlist_b = json.load(handle)
    

print(len(resultlist_b))
print(len(hitlist_b))
print(hitlist_b)

../../pilot01b2019/production/created_2019-Jan-29_15_02_20_final.json
4
4
[{'HIT': {'HITId': '30OITAWPBR8WKT016VTE67ZRJKBH9Z', 'HITTypeId': '3BIS89HFPL0BA9JSSFVWQLC6XEO1R0', 'HITGroupId': '3NVVDJT9H7SF36EZATH7618OHCHXPH', 'HITLayoutId': '3Z1ISOOYTVR8DK77D9EQ04V36BSOT2', 'Title': 'Name objects in images', 'Description': 'Look at an image and write down a name for the object in the red box', 'HITStatus': 'Assignable', 'MaxAssignments': 40, 'Reward': '0.15', 'AutoApprovalDelayInSeconds': 2592000, 'AssignmentDurationInSeconds': 900, 'QualificationRequirements': [{'QualificationTypeId': '00000000000000000040', 'Comparator': 'GreaterThanOrEqualTo', 'IntegerValues': [2000], 'RequiredToPreview': True, 'ActionsGuarded': 'PreviewAndAccept'}, {'QualificationTypeId': '000000000000000000L0', 'Comparator': 'GreaterThanOrEqualTo', 'IntegerValues': [80], 'RequiredToPreview': True, 'ActionsGuarded': 'PreviewAndAccept'}, {'QualificationTypeId': '00000000000000000071', 'Comparator': 'In', 'LocaleValues':

In [8]:
hit2images = {}
for hit in hitlist_a + hitlist_b:
    hit2images[hit['HIT']['HITId']] = hit['Images']

In [9]:
hit2images.keys()

dict_keys(['36MUZ9VAE77W39MGSAQA70L4IPAED9', '368IUKXGA6D1FRWRR1Y7V6AGFAP6PU', '3L1EFR8WWUAC6X9VB55E3MNE35A9F5', '3CMV9YRYP46NC1W0JSZPNT61W68JLF', '30OITAWPBR8WKT016VTE67ZRJKBH9Z', '3538U0YQ1GZQRVKFUW44C9KS3DOF3I', '3CVBMEMMXCLS6AC8RYVUSB8LOUO7HE', '3D4BBDG7ZI1JOJ053UE343ZP60XC35'])

In [10]:
obj2answers = {}
obj2info = {}
for assignm in resultlist_a:
    hitid = assignm['HITId']
    print(hitid)
    for a in assignm['Assignments']:
        #print(a)
        answers = a['Answers']
        for ix in range(10):
            object_id = hit2images[hitid][str(ix)][1]
            name = answers[str(ix)]['inputbox_objname-'+str(ix)]
            if object_id not in obj2answers:
                obj2answers[object_id] = {'a':Counter(),\
                                          'b':Counter(),\
                                          't':Counter()}
                obj2info[object_id] = hit2images[hitid][str(ix)]
                
            if name:
                name = name.lower()
                obj2answers[object_id]['a'][name] += 1
                obj2answers[object_id]['t'][name] += 1
                
for assignm in resultlist_b:
    hitid = assignm['HITId']
    print(hitid)
    for a in assignm['Assignments']:
        #print(a)
        answers = a['Answers']
        for ix in range(10):
            object_id = hit2images[hitid][str(ix)][1]
            name = answers[str(ix)]['inputbox_objname-'+str(ix)]
                
            if name:
                name = name.lower()
                obj2answers[object_id]['b'][name] += 1
                obj2answers[object_id]['t'][name] += 1
                
            


36MUZ9VAE77W39MGSAQA70L4IPAED9
368IUKXGA6D1FRWRR1Y7V6AGFAP6PU
3L1EFR8WWUAC6X9VB55E3MNE35A9F5
3CMV9YRYP46NC1W0JSZPNT61W68JLF
30OITAWPBR8WKT016VTE67ZRJKBH9Z
3538U0YQ1GZQRVKFUW44C9KS3DOF3I
3CVBMEMMXCLS6AC8RYVUSB8LOUO7HE
3D4BBDG7ZI1JOJ053UE343ZP60XC35


In [11]:
allrows = []
for objid in obj2answers:
    row = obj2info[objid]
    row.append(obj2answers[objid]['a'])
    row.append(obj2answers[objid]['b'])
    row.append(obj2answers[objid]['t'])
    allrows.append(row)

In [12]:
fulldf = pd.DataFrame(allrows,columns=['vg_img_id','vg_object_id','cat','synset','url','responses_a','responses_b','responses_total'])

In [13]:
fulldf

,vg_img_id,vg_object_id,cat,synset,url,responses_a,responses_b,responses_total
0,2389064,507067,ambiguous_synset,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'ottoman': 25, 'outtoman': 1, 'sofar': 1, 'po...","{'ottomon': 1, 'cushion': 2, 'ottoman': 23, 'o...","{'ottoman': 48, 'outtoman': 1, 'sofar': 1, 'po..."
1,2316226,1055203,ambiguous_synset,aircraft.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jet fighter': 2, 'jet': 18, 'aircraft': 1, '...","{'airplane': 7, 'jet': 15, 'fighter jet': 8, '...","{'jet fighter': 3, 'jet': 33, 'aircraft': 1, '..."
2,2398812,1175558,ambiguous_synset,horse.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'horse head': 6, 'horse': 28, 'horse's head':...","{'horse': 27, 'horse's head': 1, 'horse head':...","{'horse head': 12, 'horse': 55, 'horse's head'..."
3,2403052,383993,ambiguous_synset,ungulate.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'baby zebra': 1, 'zebra': 37, 'line horse': 1}","{'zebra': 36, 'baby zebra': 2, 'horse': 1, 'ch...","{'baby zebra': 3, 'zebra': 73, 'line horse': 1..."
4,2393917,1215702,singleton_category,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'sunflower': 29, 'sun flower': 4, 'flower': 5...","{'sunflower': 30, 'sun flower': 2, 'tie': 1, '...","{'sunflower': 59, 'sun flower': 6, 'flower': 9..."
5,2363125,2056940,singleton_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jacket': 22, 'man': 1, 'coat': 2, 'shirt': 1...","{'jacket': 16, 'coat': 4, 'upper body of human...","{'jacket': 38, 'man': 2, 'coat': 6, 'shirt': 1..."
6,2356851,3120857,singleton_obj,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'dandelion': 6, 'flower': 26, 'marigold': 1, ...","{'flower': 27, 'flowers': 1, 'yellow flower': ...","{'dandelion': 11, 'flower': 53, 'marigold': 1,..."
7,2405320,654642,ambiguous_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'man': 2, 'baby sling': 2, 'towel': 1, 'napki...","{'scarf': 2, 'body wrap': 2, 'carrier': 1, 'up...","{'man': 4, 'baby sling': 4, 'towel': 1, 'napki..."
8,2393669,1217850,ambiguous_category,car.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'car': 35, 'automobile': 1, 'vehicle': 2, 'ca...","{'car': 33, 'salon car': 1, 'automobile': 4, '...","{'car': 68, 'automobile': 5, 'vehicle': 3, 'ca..."
9,2330865,2895016,singleton_category,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'dirt bike': 1, 'mat': 15, 'floor mat': 4, 'c...","{'mat': 14, 'floor mat': 9, 'floor matt': 1, '...","{'dirt bike': 1, 'mat': 29, 'floor mat': 13, '..."


In [14]:
def get_mae(row):
    a = []
    b = []
    for name in row['responses_total']:
        a.append(row['responses_a'][name])
        b.append(row['responses_b'][name])
        
    #print(a)
    #print(b)
    #r = spearmanr(np.array(a),np.array(b))
    #print(r[0])
    
    mean_abs_error = np.mean(np.abs((np.array(a) - np.array(b))))
    
    #print(mean_abs_error)
    return mean_abs_error
    

In [15]:
def get_agreement(rdict):
   
    vec = np.array([rdict[key] for key in rdict if rdict[key] > 1])
    vec_rel = vec/(np.sum(vec))
    #print(vec)
    #print(vec_rel)
    agr = np.sum(vec_rel * np.log2(1/vec_rel))
    
    return agr

In [16]:
get_agreement(fulldf.iloc[0]['responses_total'])

1.1489694603653073

In [17]:
fulldf['agr_total'] = fulldf['responses_total'].apply(lambda x: get_agreement(x))
fulldf['agr_a'] = fulldf['responses_a'].apply(lambda x: get_agreement(x))
fulldf['agr_b'] = fulldf['responses_b'].apply(lambda x: get_agreement(x))

In [18]:
fulldf['mae_a_b'] = [get_mae(row) for ix,row in fulldf.iterrows()]

In [19]:
fulldf

,vg_img_id,vg_object_id,cat,synset,url,responses_a,responses_b,responses_total,agr_total,agr_a,agr_b,mae_a_b
0,2389064,507067,ambiguous_synset,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'ottoman': 25, 'outtoman': 1, 'sofar': 1, 'po...","{'ottomon': 1, 'cushion': 2, 'ottoman': 23, 'o...","{'ottoman': 48, 'outtoman': 1, 'sofar': 1, 'po...",1.148969,0.716726,0.850326,0.916667
1,2316226,1055203,ambiguous_synset,aircraft.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jet fighter': 2, 'jet': 18, 'aircraft': 1, '...","{'airplane': 7, 'jet': 15, 'fighter jet': 8, '...","{'jet fighter': 3, 'jet': 33, 'aircraft': 1, '...",2.065899,1.794737,2.014719,1.294118
2,2398812,1175558,ambiguous_synset,horse.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'horse head': 6, 'horse': 28, 'horse's head':...","{'horse': 27, 'horse's head': 1, 'horse head':...","{'horse head': 12, 'horse': 55, 'horse's head'...",1.212842,0.672295,1.040852,0.900000
3,2403052,383993,ambiguous_synset,ungulate.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'baby zebra': 1, 'zebra': 37, 'line horse': 1}","{'zebra': 36, 'baby zebra': 2, 'horse': 1, 'ch...","{'baby zebra': 3, 'zebra': 73, 'line horse': 1...",0.239874,0.000000,0.297472,1.000000
4,2393917,1215702,singleton_category,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'sunflower': 29, 'sun flower': 4, 'flower': 5...","{'sunflower': 30, 'sun flower': 2, 'tie': 1, '...","{'sunflower': 59, 'sun flower': 6, 'flower': 9...",1.075361,1.024478,0.803072,1.000000
5,2363125,2056940,singleton_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jacket': 22, 'man': 1, 'coat': 2, 'shirt': 1...","{'jacket': 16, 'coat': 4, 'upper body of human...","{'jacket': 38, 'man': 2, 'coat': 6, 'shirt': 1...",1.443496,0.871096,1.251629,1.222222
6,2356851,3120857,singleton_obj,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'dandelion': 6, 'flower': 26, 'marigold': 1, ...","{'flower': 27, 'flowers': 1, 'yellow flower': ...","{'dandelion': 11, 'flower': 53, 'marigold': 1,...",0.896113,0.978016,0.625262,1.000000
7,2405320,654642,ambiguous_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'man': 2, 'baby sling': 2, 'towel': 1, 'napki...","{'scarf': 2, 'body wrap': 2, 'carrier': 1, 'up...","{'man': 4, 'baby sling': 4, 'towel': 1, 'napki...",3.357052,3.083587,2.789866,1.093750
8,2393669,1217850,ambiguous_category,car.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'car': 35, 'automobile': 1, 'vehicle': 2, 'ca...","{'car': 33, 'salon car': 1, 'automobile': 4, '...","{'car': 68, 'automobile': 5, 'vehicle': 3, 'ca...",0.585927,0.303375,0.494183,1.500000
9,2330865,2895016,singleton_category,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'dirt bike': 1, 'mat': 15, 'floor mat': 4, 'c...","{'mat': 14, 'floor mat': 9, 'floor matt': 1, '...","{'dirt bike': 1, 'mat': 29, 'floor mat': 13, '...",2.216130,1.750632,1.954940,1.315789


In [20]:
error = 0
total = 0
for ix,row in fulldf.iterrows():
    a = []
    b = []
    for name in row['responses_total']:
        a.append(row['responses_a'][name])
        b.append(row['responses_b'][name])
        
    #print(a)
    #print(b)
    #r = spearmanr(np.array(a),np.array(b))
    #print(r[0])
    
    error += np.sum(np.abs((np.array(a) - np.array(b))))
    total += len(a)
    
error/total
    

1.2663934426229508

In [22]:
fulldf.to_csv("40assignments_results.csv",index=False)

In [23]:
np.mean(fulldf['agr_total'])

1.2750091671857429

In [24]:
np.mean(fulldf['agr_a'])

0.9646796526791052

In [25]:
np.mean(fulldf['agr_b'])

1.154019773767479

In [26]:
from scipy.stats import ttest_rel

In [27]:
ttest_rel(fulldf['agr_total'],fulldf['agr_a'])

Ttest_relResult(statistic=9.0128668329964885, pvalue=4.4586532865321303e-11)

In [28]:
np.mean(fulldf[fulldf['cat'] == 'ambiguous_category']['agr_total'])

1.611126818690861

In [29]:
np.mean(fulldf[fulldf['cat'] == 'singleton_category']['agr_total'])

1.097399921257097

In [30]:
for c in set(list(fulldf['synset'])):
    print(c,np.mean(fulldf[fulldf['synset'] == c]['agr_total']))

boat.n.01 1.3659889742138787
furnishing.n.02 1.194764191609065
ship.n.01 1.5457830510722934
ungulate.n.01 0.997363485614458
horse.n.01 0.6064211304180316
overgarment.n.01 2.026793364272883
headdress.n.01 1.075446657660468
neckwear.n.01 1.7384996943383006
train.n.01 0.0
bird.n.01 1.015314045477072
flower.n.01 0.9857369900484061
car.n.01 1.7460561484082942
aircraft.n.01 1.7446986563020304


In [90]:
spell = SpellChecker()
Lem = WordNetLemmatizer()

In [91]:
def check_spelling(answers):
    d = answers
    misspelled = spell.unknown(d.keys())
    ok = {w:d[w] for w in d.keys() if not w in misspelled}
    #print(misspelled)
    #print(ok)
    
    for mis in misspelled:
        #print("Checking:",mis)
        cands = spell.candidates(mis)
        #print("Cand:",cands)
        corrected = False
        for c in ok:
            if c in cands:
                #print("YAY")
                ok[c] += d[mis]
                corrected = True
                break
        if not corrected:
            ok[mis] = d[mis]
            
    clean_answers = Counter()
    for w in ok:
        lem_w = Lem.lemmatize(w)
        clean_answers[lem_w] += ok[w]
        
    return clean_answers
        
            
    

In [93]:
obj2clean_answers = {objid:check_spelling(obj2answers[objid]['t']) for objid in obj2answers}


In [95]:
fulldf['cleaned_responses'] = [obj2clean_answers[objid] for objid in obj2answers]

In [96]:
fulldf

,vg_img_id,vg_object_id,cat,synset,url,responses_a,responses_b,responses_total,agr_total,agr_a,agr_b,mae_a_b,cleaned_responses
0,2389064,507067,ambiguous_synset,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'ottoman': 25, 'outtoman': 1, 'sofar': 1, 'po...","{'ottomon': 1, 'cushion': 2, 'ottoman': 23, 'o...","{'ottoman': 48, 'outtoman': 1, 'sofar': 1, 'po...",1.148969,0.716726,0.850326,0.916667,"{'ottoman': 56, 'dog': 1, 'bed': 2, 'footrest'..."
1,2316226,1055203,ambiguous_synset,aircraft.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jet fighter': 2, 'jet': 18, 'aircraft': 1, '...","{'airplane': 7, 'jet': 15, 'fighter jet': 8, '...","{'jet fighter': 3, 'jet': 33, 'aircraft': 1, '...",2.065899,1.794737,2.014719,1.294118,"{'jet': 33, 'aircraft': 1, 'plane': 6, 'airpla..."
2,2398812,1175558,ambiguous_synset,horse.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'horse head': 6, 'horse': 28, 'horse's head':...","{'horse': 27, 'horse's head': 1, 'horse head':...","{'horse head': 12, 'horse': 55, 'horse's head'...",1.212842,0.672295,1.040852,0.900000,"{'horse': 56, 'face': 1, 'van': 1, 'head': 3, ..."
3,2403052,383993,ambiguous_synset,ungulate.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'baby zebra': 1, 'zebra': 37, 'line horse': 1}","{'zebra': 36, 'baby zebra': 2, 'horse': 1, 'ch...","{'baby zebra': 3, 'zebra': 73, 'line horse': 1...",0.239874,0.000000,0.297472,1.000000,"{'zebra': 73, 'horse': 1, 'line horse': 1, 'ba..."
4,2393917,1215702,singleton_category,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'sunflower': 29, 'sun flower': 4, 'flower': 5...","{'sunflower': 30, 'sun flower': 2, 'tie': 1, '...","{'sunflower': 59, 'sun flower': 6, 'flower': 9...",1.075361,1.024478,0.803072,1.000000,"{'sunflower': 68, 'flower': 10, 'tie': 1}"
5,2363125,2056940,singleton_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'jacket': 22, 'man': 1, 'coat': 2, 'shirt': 1...","{'jacket': 16, 'coat': 4, 'upper body of human...","{'jacket': 38, 'man': 2, 'coat': 6, 'shirt': 1...",1.443496,0.871096,1.251629,1.222222,"{'jacket': 39, 'man': 2, 'coat': 6, 'shirt': 1..."
6,2356851,3120857,singleton_obj,flower.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'dandelion': 6, 'flower': 26, 'marigold': 1, ...","{'flower': 27, 'flowers': 1, 'yellow flower': ...","{'dandelion': 11, 'flower': 53, 'marigold': 1,...",0.896113,0.978016,0.625262,1.000000,"{'dandelion': 11, 'flower': 55, 'marigold': 1,..."
7,2405320,654642,ambiguous_category,overgarment.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'man': 2, 'baby sling': 2, 'towel': 1, 'napki...","{'scarf': 2, 'body wrap': 2, 'carrier': 1, 'up...","{'man': 4, 'baby sling': 4, 'towel': 1, 'napki...",3.357052,3.083587,2.789866,1.093750,"{'man': 4, 'towel': 1, 'napkin': 4, 'hand': 3,..."
8,2393669,1217850,ambiguous_category,car.n.01,http://www.coli.uni-saarland.de/~carina/object...,"{'car': 35, 'automobile': 1, 'vehicle': 2, 'ca...","{'car': 33, 'salon car': 1, 'automobile': 4, '...","{'car': 68, 'automobile': 5, 'vehicle': 3, 'ca...",0.585927,0.303375,0.494183,1.500000,"{'car': 69, 'automobile': 5, 'vehicle': 3, 'sh..."
9,2330865,2895016,singleton_category,furnishing.n.02,http://www.coli.uni-saarland.de/~carina/object...,"{'dirt bike': 1, 'mat': 15, 'floor mat': 4, 'c...","{'mat': 14, 'floor mat': 9, 'floor matt': 1, '...","{'dirt bike': 1, 'mat': 29, 'floor mat': 13, '...",2.216130,1.750632,1.954940,1.315789,"{'mat': 29, 'carpet': 2, 'motorcycle': 5, 'bik..."


In [97]:
fulldf['agr_cleaned'] = fulldf['cleaned_responses'].apply(lambda x: get_agreement(x))

In [47]:
for c in set(list(fulldf['synset'])):
    print(c,np.mean(fulldf[fulldf['synset'] == c]['agr_total']))

2

In [98]:
np.mean(fulldf['agr_cleaned'])

1.2491225067869518

In [99]:
np.mean(fulldf['agr_total'])

1.2750091671857429

In [82]:
spell.known(spell.edit_distance_2('pouf'))

{'aoun',
 'auf',
 'boeuf',
 'bou',
 'bout',
 'buf',
 'chouf',
 'coif',
 'cou',
 'coum',
 'coun',
 'coup',
 'cour',
 'cous',
 'diouf',
 'doff',
 'domf',
 'dorf',
 'doug',
 'dour',
 'fou',
 'foul',
 'four',
 'goff',
 'golf',
 'goof',
 'gout',
 'hof',
 'hoff',
 'hoof',
 'hou',
 'hour',
 'hous',
 'iou',
 'ious',
 'jour',
 'loaf',
 'lou',
 'loud',
 'lout',
 'moue',
 'neuf',
 'norf',
 'nou',
 'noun',
 'nour',
 'nous',
 'oaf',
 'of',
 'off',
 'olf',
 'omf',
 'opus',
 'orf',
 'osf',
 'ou',
 'oui',
 'oup',
 'our',
 'out',
 'paf',
 'pau',
 'paul',
 'pcf',
 'pdf',
 'pdgf',
 'peu',
 'peut',
 'peux',
 'pf',
 'pff',
 'phu',
 'piaf',
 'pious',
 'pius',
 'plf',
 'plug',
 'plum',
 'plus',
 'plut',
 'pmsf',
 'pnsf',
 'po',
 'poa',
 'poc',
 'pod',
 'pods',
 'poe',
 'poem',
 'poet',
 'pogo',
 'pohl',
 'poi',
 'pois',
 'poke',
 'poky',
 'pol',
 'pole',
 'poli',
 'poll',
 'polo',
 'poly',
 'pom',
 'pomp',
 'poms',
 'pon',
 'pond',
 'pong',
 'pons',
 'pont',
 'pony',
 'poo',
 'poof',
 'poofs',
 'pooh',
 'poo

In [81]:
spell.edit_distance_2['pouffe']

15

In [103]:
for c in set(list(fulldf['synset'])):
    print(c,"%.3f"%np.mean(fulldf[fulldf['synset'] == c]['agr_total']))

boat.n.01 1.366
furnishing.n.02 1.195
ship.n.01 1.546
ungulate.n.01 0.997
horse.n.01 0.606
overgarment.n.01 2.027
headdress.n.01 1.075
neckwear.n.01 1.738
train.n.01 0.000
bird.n.01 1.015
flower.n.01 0.986
car.n.01 1.746
aircraft.n.01 1.745


In [105]:
np.mean(fulldf['agr_total'])

1.2750091671857429

In [89]:
Lem.lemmatize('fighter jet (airplane)')

'fighter jet (airplane)'

In [110]:
for d in fulldf[fulldf['synset'] == 'train.n.01']['responses_total']:
    print(d)

Counter({'train': 72, 'bus': 1, 'bullet train': 1, 'tram': 1, 'floor': 1, 'high-speed train': 1, 'fast train': 1})
